In [1]:
from utils import*

In [2]:
import numpy as np
import cvxpy as cp
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cvxpy'

In [ ]:
source_dir='./'

In [ ]:
covid_features=sio.loadmat(os.path.join(source_dir,'covid.mat')) 
covid_features=covid_features['covid'] 

normal_features=sio.loadmat(os.path.join(source_dir,'normal.mat')) 
normal_features=normal_features['normal']  

pneumonia_features=sio.loadmat(os.path.join(source_dir,'pneumonia.mat')) 
pneumonia_features=pneumonia_features['pneumonia']  

In [ ]:
X=np.concatenate((covid_features[:,:-1],normal_features[:,:-1],pneumonia_features[:,:-1]), axis=0)#inputs
y=np.concatenate((covid_features[:,-1],normal_features[:,-1],pneumonia_features[:,-1]), axis=0)#target labels

In [ ]:
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [ ]:
transformer = KernelPCA(n_components=64, kernel='linear')
X = transformer.fit_transform(X)

In [ ]:
y = to_categorical(y)

In [ ]:
m,n = np.shape(X)
W = cp.Variable(n)   # Weight matrix
Psi = cp.Variable(m)  # Slack variable
b = cp.Variable()  # Gamma term
C = 0.01  # Control/Regularization paramters

In [ ]:
# Soft-margin SVM
# Objective function for softmargin svm is 0.5W'W + Ce'Psi
# cvxpy command for computing the transpose of a matrix is cp.transpose(W)
objective = cp.Minimize(0.5 * cp.square(cp.norm(W)) + C * cp.sum(Psi))
# Constraint y(W'x+b)+Psi-1>=0; Psi>=0
constraints = [cp.matmul(y, X @ W + b)-1 + Psi >= 0, Psi >= 0]

prob = cp.Problem(objective, constraints)
prob.solve()
print("Status of optimization:", prob.status)

print("Optimal value of W:", W.value)
print("Optimal value of b:", b.value)

print("Constraint_0:", constraints[0])
print("Constraint_1:", constraints[1]) # second constraint is available for soft-margin svm

print("Dual value for the first constraint:", constraints[0].dual_value) # Value of the lagrangian multiplier
print("Dual value for the second constraint:", constraints[1].dual_value) # second constraint is available for soft-margin svm